# 3rd Level Model Structure: Everything (Linking Stages)

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

In [2]:
from pyomo import environ as pe
from modules.global_set import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,2)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Linking Stage Variables

### Vapor Between Reactive Stages

In [6]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [7]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [8]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [9]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

# Fixing Redundent Stream Variables

In [10]:
# condenser
model.condenser.VLE_block.n_ave.fix(58)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [11]:
# 'reboiler' fixing last stage V_in

model.reactive[model.TRAY.last()].V['in'].fix(0)
for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

In [12]:
check_DOF(pe,model)

Active Equality Constraints:	 3163
Active Inequality Constraints:	 0
Active Variables:		 3357
Fixed Variables:		 169
DOF:				 25


# Load Operating Parameters

In [27]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].setub(12)
# model.condenser.L['P'].fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(1500)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(58)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(0.3333)
    model.reactive[j].z['H2'].fix(0.6666)
    model.reactive[j].z['C30H62'].fix(0.0001)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(240+273.15)

In [21]:
model.reactive[1].Q_main.fix(0)

In [14]:
check_DOF(pe,model)

Active Equality Constraints:	 3163
Active Inequality Constraints:	 0
Active Variables:		 3357
Fixed Variables:		 191
DOF:				 3


In [15]:
model.obj = pe.Objective(expr = model.reactive[1].T+model.reactive[2].T ,sense=pe.maximize)

In [16]:
opt = pe.SolverFactory('ipopt')
opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
# opt.options['nlp_scaling_method'] = None
# opt.options['constr_viol_tol'] = 1e-7
# opt.options['acceptable_constr_viol_tol'] = 1e-7
opt.options['max_iter'] = 7000
# opt.options['dual_inf_tol'] = '+inf'
# opt.options['acceptable_dual_inf_tol'] = '+inf'

In [17]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [31]:
results = opt.solve(model,tee=True)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma86                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma86.

Number of nonzeros in equality constraint Jacobian...:    10402
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.....

  87 -1.0259778e+03 1.94e-01 1.17e+06  -1.0 1.64e+02    -  1.01e-01 8.26e-04h  4
  88 -1.0260162e+03 1.94e-01 5.97e+06  -1.0 1.14e+02    -  7.39e-01 1.56e-03h  4
  89 -1.0260508e+03 1.93e-01 6.22e+07  -1.0 1.07e+02    -  1.39e-01 1.73e-03h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90 -1.0260809e+03 1.93e-01 4.50e+09  -1.0 1.06e+02    -  9.06e-01 1.53e-03h  4
  91 -1.0261342e+03 1.92e-01 5.89e+10  -1.0 1.06e+02    -  1.39e-01 2.72e-03h  3
  92 -1.0261739e+03 1.92e-01 7.03e+12  -1.0 1.04e+02    -  1.00e+00 2.04e-03h  3
  93 -1.0262044e+03 1.92e-01 7.92e+12  -1.0 1.03e+02    -  1.43e-01 1.57e-03h  3
  94 -1.0262186e+03 1.91e-01 9.53e+14  -1.0 8.01e+01    -  1.00e+00 3.93e-03h  1
  95 -1.0262471e+03 1.91e-01 9.54e+14  -1.0 1.93e+02    -  1.11e-01 4.02e-04h  2
  96 -1.0262896e+03 1.90e-01 9.68e+14  -1.0 2.11e+02    -  1.84e-01 6.75e-04H  1
  97 -1.0262997e+03 1.90e-01 1.44e+15  -1.0 3.30e+02    -  6.99e-02 1.11e-04h  1
  98 -1.0262997e+03 1.90e-01

 179 -1.0166878e+03 2.46e+00 1.82e+14  -1.0 1.26e+02    -  5.44e-03 6.07e-02w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180 -1.0258595e+03 5.29e-03 1.28e+14  -1.0 6.93e+02    -  1.39e-02 1.50e-04h 10
 181 -1.0258559e+03 5.29e-03 1.28e+14  -1.0 1.06e+02    -  1.00e+00 1.50e-04h 11
 182 -1.0258522e+03 5.29e-03 1.28e+14  -1.0 1.06e+02    -  1.26e-02 1.50e-04h 11
 183 -1.0258486e+03 5.29e-03 1.28e+14  -1.0 1.06e+02    -  3.80e-01 1.50e-04h 11
 184 -1.0258449e+03 5.29e-03 1.28e+14  -1.0 1.06e+02    -  1.44e-02 1.50e-04h 11
 185 -1.0258413e+03 5.29e-03 1.28e+14  -1.0 1.06e+02    -  1.00e+00 1.50e-04h 11
 186 -1.0258376e+03 5.28e-03 1.28e+14  -1.0 1.06e+02    -  1.32e-02 1.49e-04h 11
 187 -1.0258340e+03 5.28e-03 1.28e+14  -1.0 1.06e+02    -  3.79e-01 1.49e-04h 11
 188 -1.0258304e+03 5.28e-03 1.28e+14  -1.0 1.06e+02    -  1.49e-02 1.49e-04h 11
 189 -1.0258267e+03 5.28e-03 1.28e+14  -1.0 1.06e+02    -  1.00e+00 1.49e-04h 11
iter    objective    inf_pr 

 271 -1.0252017e+03 5.20e-03 1.28e+14  -1.0 8.79e-08  19.1 3.21e-02 2.67e-04h  9
 272 -1.0251912e+03 5.20e-03 1.28e+14  -1.0 5.00e+01    -  2.72e-01 2.66e-04h 10
 273 -1.0251808e+03 5.20e-03 1.28e+14  -1.0 5.00e+01    -  2.66e-02 2.66e-04h 10
 274 -1.0251703e+03 5.20e-03 1.28e+14  -1.0 5.00e+01    -  1.00e+00 2.66e-04h 10
 275 -1.0251599e+03 5.20e-03 1.28e+14  -1.0 5.00e+01    -  2.54e-02 2.65e-04h 10
 276 -1.0251495e+03 5.20e-03 1.28e+14  -1.0 5.00e+01    -  9.91e-02 2.65e-04h 10
 277 -1.0251391e+03 5.19e-03 1.28e+14  -1.0 5.00e+01    -  3.85e-02 2.64e-04h 10
 278 -1.0251287e+03 5.19e-03 1.28e+14  -1.0 5.00e+01    -  1.42e-01 2.64e-04h 10
 279 -1.0251183e+03 5.19e-03 1.28e+14  -1.0 5.01e+01    -  3.13e-02 2.64e-04h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 280 -1.0251080e+03 5.19e-03 1.28e+14  -1.0 5.01e+01    -  1.00e+00 2.63e-04h 10
 281 -1.0198203e+03 1.13e-01 4.50e+13  -1.0 5.00e+01    -  2.68e-02 1.34e-01w  1
 282 -1.0182781e+03 2.07e-01

 364 -1.0244926e+03 5.12e-03 1.28e+14  -1.0 4.97e+01    -  4.84e-02 2.30e-04h 10
 365 -1.0244834e+03 5.11e-03 1.28e+14  -1.0 4.97e+01    -  7.93e-01 2.29e-04h 10
 366 -1.0244743e+03 5.11e-03 1.28e+14  -1.0 4.96e+01    -  4.04e-02 2.29e-04h 10
 367 -1.0244651e+03 5.11e-03 1.28e+14  -1.0 4.97e+01    -  1.00e+00 2.28e-04h 10
 368 -1.0244559e+03 5.11e-03 1.28e+14  -1.0 4.96e+01    -  4.08e-02 2.28e-04h 10
 369 -1.0244468e+03 5.11e-03 1.28e+14  -1.0 4.97e+01    -  1.99e-01 2.28e-04h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370 -1.0244376e+03 5.11e-03 1.28e+14  -1.0 4.97e+01    -  5.13e-02 2.27e-04h 10
 371 -1.0244285e+03 5.11e-03 1.28e+14  -1.0 4.97e+01    -  5.34e-01 2.27e-04h 10
 372 -1.0197559e+03 9.49e-02 6.05e+13  -1.0 4.97e+01    -  4.28e-02 1.16e-01w  1
 373 -1.0182621e+03 1.99e-01 4.09e+14  -1.0 6.03e+01    -  8.85e-02 2.92e-02w  1
 374 -1.0182754e+03 1.99e-01 1.64e+17  -1.0 3.44e+02    -  8.42e-02 3.61e-04w  1
 375 -1.0244193e+03 5.11e-03

...

1385 -1.0194339e+03 4.55e-03 1.52e+14  -1.0 5.18e+01    -  4.71e-01 1.00e-04h  9
1386 -1.0183356e+03 9.76e-02 2.76e+15  -1.0 5.18e+01    -  1.06e-01 2.55e-02w  1
1387 -1.0180268e+03 9.83e-02 4.62e+16  -1.0 7.73e+02    -  1.56e-02 1.40e-03w  1
1388 -1.0191259e+03 2.11e-01 2.93e+17  -1.0 3.56e+03    -  8.44e-03 1.46e-03w  1
1389 -1.0194296e+03 4.55e-03 1.53e+14  -1.0 4.30e+02    -  1.06e-01 9.96e-05h  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1390 -1.0194253e+03 4.54e-03 1.53e+14  -1.0 5.18e+01    -  1.00e+00 9.93e-05h  9
1391 -1.0194211e+03 4.54e-03 1.53e+14  -1.0 5.17e+01    -  9.26e-02 9.89e-05h  9
1392 -1.0194168e+03 4.54e-03 1.53e+14  -1.0 5.17e+01    -  4.68e-01 9.86e-05h  9
1393 -1.0194126e+03 4.54e-03 1.53e+14  -1.0 5.17e+01    -  1.05e-01 9.82e-05h  9
1394 -1.0194084e+03 4.54e-03 1.53e+14  -1.0 5.17e+01    -  1.00e+00 9.79e-05h  9
1395 -1.0194042e+03 4.54e-03 1.53e+14  -1.0 5.17e+01    -  9.19e-02 9.75e-05h  9
1396 -1.0194000e+03 4.54e-03

1478 -1.0179786e+03 7.66e-02 6.07e+16  -1.0 1.14e+03    -  1.07e-02 1.13e-03w  1
1479 -1.0192266e+03 2.81e-01 3.37e+17  -1.0 2.86e+03    -  1.05e-02 2.10e-03w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1480 -1.0191652e+03 4.52e-03 1.59e+14  -1.0 2.84e-02  16.8 9.99e-02 7.75e-05h  8
1481 -1.0191619e+03 4.52e-03 1.59e+14  -1.0 5.16e+01    -  1.00e+00 7.72e-05h  9
1482 -1.0191586e+03 4.52e-03 1.59e+14  -1.0 5.16e+01    -  8.11e-02 7.70e-05h  9
1483 -1.0191553e+03 4.52e-03 1.59e+14  -1.0 5.15e+01    -  3.22e-01 7.67e-05h  9
1484 -1.0191521e+03 4.52e-03 1.59e+14  -1.0 5.15e+01    -  1.00e-01 7.64e-05h  9
1485 -1.0191488e+03 4.52e-03 1.59e+14  -1.0 5.15e+01    -  1.00e+00 7.61e-05h  9
1486 -1.0191456e+03 4.52e-03 1.59e+14  -1.0 5.15e+01    -  8.05e-02 7.59e-05h  9
1487 -1.0191424e+03 4.52e-03 1.59e+14  -1.0 5.15e+01    -  3.23e-01 7.56e-05h  9
1488 -1.0191392e+03 4.52e-03 1.59e+14  -1.0 5.15e+01    -  9.95e-02 7.53e-05h  9
1489 -1.0191360e+03 4.52e-03

1571 -1.0189585e+03 4.50e-03 1.66e+14  -1.0 2.81e+02    -  7.23e-02 6.02e-05h  8
1572 -1.0189559e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  2.85e-01 6.00e-05h  9
1573 -1.0189532e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  9.13e-02 5.97e-05h  9
1574 -1.0189506e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  1.00e+00 5.95e-05h  9
1575 -1.0189480e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  7.18e-02 5.93e-05h  9
1576 -1.0189454e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  2.83e-01 5.91e-05h  9
1577 -1.0189428e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  9.08e-02 5.89e-05h  9
1578 -1.0189402e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  1.00e+00 5.87e-05h  9
1579 -1.0189376e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  7.14e-02 5.85e-05h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1580 -1.0189350e+03 4.50e-03 1.66e+14  -1.0 5.14e+01    -  2.82e-01 5.82e-05h  9
1581 -1.0182770e+03 5.48e-02 7.80e+15  -1.0 5.14e+01    -  9.03e-02 1.49e-02w  1
1582 -1.0179778e+03 5.50e-02

1664 -1.0187876e+03 4.48e-03 1.73e+14  -1.0 5.12e+01    -  6.50e-02 4.63e-05h  9
1665 -1.0187855e+03 4.48e-03 1.73e+14  -1.0 5.12e+01    -  2.63e-01 4.61e-05h  9
1666 -1.0187834e+03 4.48e-03 1.73e+14  -1.0 5.12e+01    -  8.26e-02 4.60e-05h  9
1667 -1.0187813e+03 4.48e-03 1.74e+14  -1.0 5.12e+01    -  1.00e+00 4.58e-05h  9
  Signal handler called from  /anaconda/lib/python3.6/subprocess.py _try_wait 1404
  Waiting...
  Signaled process 32252 with signal 2
ERROR: "[base]/site-packages/pyomo/opt/base/solvers.py", 616, solve
	Solver (ipopt) returned non-zero return code (-1)
ERROR: "[base]/site-packages/pyomo/opt/base/solvers.py", 619, solve
	See the solver log above for diagnostic information.


ApplicationError: Solver (ipopt) did not exit normally

In [29]:
update_dual(pe,model)

In [30]:
beautify2(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
		T		 Q			 V			 L
               30.00 	       -77.0620552219 	         0.2957928877 	         1.4394511406
              240.00 	         0.0000000000 	         2.1366709315 	         0.2267605822
              240.00 	       -40.5542666271 	         0.5582149086 	         0.1961564444
----------------------------------------------------------------------------------------------------
Top
V	 0.2957928876508373
L	 0.0
W	 0.401426913105738
----------------------------------------------------------------------------------------------------
Bottom
L	 0.19615644444230795
----------------------------------------------------------------------------------------------------
Condenser:	Vapor		Liquid		Last Stage	Vapor		Liquid
H2 		51.510%		2.007%		 H2       	34.544%		3.049%
CO 		8.362%		0.602%		 CO       	4.852%		0.489%
CO2 		17.193%		10.436%		 CO2      	10.082%		2.561%
H2O

In [ ]:
model.reactive[1].kinetics_block.r_FT_total.value

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/1_stage_condenser_n58_tmp.pickle','wb') as f:
#     pickle.dump(results,f)

### Check non-active constraints (dual = 0)

In [ ]:
for i in model.dual:
    if abs(model.dual[i]) == 0:
        print('{:40s}'.format(str(i)),'\t\t',model.dual[i])

# Iterative Data Analysis

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = 1 ,sense=pe.minimize)
model.reactive[1].T.fixed = True
model.reactive[1].Q_main.fixed = False
model.condenser.L['out'].setub(None)

In [ ]:
check_DOF(pe,model)

In [ ]:
update_dual(pe,model)

In [ ]:
Trange = np.linspace(model.reactive[1].T.value,220+273.15,40)

In [ ]:
rf_data = {}
rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; rf_data['Re'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []

for i in model.reactive[1].r_total_comp:
    rf_data['r'][i] = []

for t in Trange:
    model.reactive[1].T.fix(t)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    print('Solved, Solution T = {} K'.format(model.reactive[1].T.value))
    rf_data['T'].append(model.reactive[1].T.value)
    rf_data['Q'].append(model.reactive[1].Q_main.value)
    rf_data['V'].append(model.condenser.V['out'].value)
    rf_data['L'].append(model.condenser.L['P'].value)
    rf_data['Re'].append(model.condenser.L['out'].value)

    for i in model.reactive[1].r_total_comp:
        rf_data['r'][i].append(model.reactive[1].r_total_comp[i].value)


    rf_data['y_H2O'].append(model.reactive[1].y['H2O'].value)
    rf_data['y_CO'].append(model.reactive[1].y['CO'].value)
    rf_data['y_H2'].append(model.reactive[1].y['H2'].value)
    rf_data['y_CO2'].append(model.reactive[1].y['CO2'].value)
    rf_data['r_WGS'].append(model.reactive[1].kinetics_block.r_WGS.value)
    rf_data['r_FT'].append(model.reactive[1].kinetics_block.r_FT_total.value)

In [ ]:
list(zip(rf_data['T'],rf_data['Re'],rf_data['L']))

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
line1 = ax1.plot(rf_data['T'],rf_data['Q'], 'b-')
ax1.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax1.set_ylabel('Q (MW)', color='b',fontsize=18)
ax1.tick_params('y', colors='b',labelsize=18)
ax1.tick_params('x', colors='k',labelsize=18)
ax1.grid()

ax2 = ax1.twinx()
line2 = ax2.plot(rf_data['T'],rf_data['r_FT'], 'r')
line3 = ax2.plot(rf_data['T'],rf_data['r_WGS'], 'r--')
line4 = ax2.plot(rf_data['T'],rf_data['V'], 'k')
line5 = ax2.plot(rf_data['T'],rf_data['L'], 'k--')
line6 = ax2.plot(rf_data['T'],rf_data['L'], 'k--')
ax2.set_ylabel('$r_{FT}$: (kmol/s Carbon Number)', color='r',fontsize=18)
ax2.tick_params('y', colors='r',labelsize=18)

ax1.legend(line1+line2+line3+line4+line5,['Q','$r_{FT}$','$r_{WGS}$','V','L'],fontsize=18,loc=5)
ax1.set_title('Heat Removal Rate and Reaction Rate vs T',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['y_CO'])
ax.plot(rf_data['T'],rf_data['y_H2'])
ax.plot(rf_data['T'],rf_data['y_CO2'])
ax.plot(rf_data['T'],rf_data['y_H2O'])

ax.legend(['$y_{CO}$','$y_{H2}$','$y_{CO2}$','$y_{H2O}$'],fontsize=18)
ax.set_title('Gas Phase Inorganic Composition vs T',fontsize=18)

ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['r']['CO'])
ax.plot(rf_data['T'],rf_data['r']['H2'])
ax.plot(rf_data['T'],rf_data['r']['CO2'])
ax.plot(rf_data['T'],rf_data['r']['H2O'])

ax.legend(['$r_{CO}$','$r_{H2}$','$r_{CO2}$','$r_{H2O}$'],fontsize=18)
ax.set_title('Inorganic Reaction Rate vs T',fontsize=18)

ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
reaction_data = trans_product_mole(rf_data['r'])['unscaled']
reaction_data_scaled = trans_product_mole(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data_scaled['c1'], reaction_data_scaled['c2'],reaction_data_scaled['c3'],reaction_data_scaled['c4'],\
            reaction_data_scaled['napha'],reaction_data_scaled['gasoline'],reaction_data_scaled['diesel'],reaction_data_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mole) vs T',fontsize=18)
ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data['c1'], reaction_data['c2'],reaction_data['c3'],reaction_data['c4'],\
            reaction_data['napha'],reaction_data['gasoline'],reaction_data['diesel'],reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (Mole) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
mass_reaction_data = trans_product_mass(rf_data['r'])['unscaled']
mass_reaction_scaled = trans_product_mass(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_scaled['c1'], mass_reaction_scaled['c2'],mass_reaction_scaled['c3'],mass_reaction_scaled['c4'],\
            mass_reaction_scaled['napha'],mass_reaction_scaled['gasoline'],mass_reaction_scaled['diesel'],mass_reaction_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mass) vs T',fontsize=18)
ax.set_ylabel('Mass Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_data['c1'], mass_reaction_data['c2'],mass_reaction_data['c3'],mass_reaction_data['c4'],\
            mass_reaction_data['napha'],mass_reaction_data['gasoline'],mass_reaction_data['diesel'],mass_reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (mass) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kg/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
model.reactive[1].V.pprint()